In [ ]:
pip install fastparquet pyarrow

In [1]:
# Librerias
import os
import boto3
import pandas as pd

In [4]:
# Cliente textract
textract_client = boto3.client('textract')

In [5]:
#--------------------------------------------------------------------
#         PROCESO AUTOMATIZADO
#--------------------------------------------------------------------

In [6]:
# Función que extrae el texto de una imagen
def text_from_image(image_path, textract_client):
    try:
        with open(image_path, 'rb') as image:
            response = textract_client.detect_document_text(Document={'Bytes': image.read()})
        text = ''
        for item in response['Blocks']:
            if item['BlockType'] == 'LINE':
                text += item['Text'] + '\n'
        return text
    except Exception as e:
        print(f"Error al procesar la imagen {image_path}: {e}")
        return None

In [7]:
# Prueba de la función text_from_image
#image_path = '/home/ec2-user/SageMaker/genebanks/data/2_images/Germplasm collection-Colombia-1990/page_1.png'
#text = text_from_image(image_path, textract_client)

In [8]:
#text

In [9]:
# Función que guarda todos los textos extraidos de las imagenes de un documento
# en un archivo parquet
def process_pdf_folder(folder_path, textract_client, output_path):
    all_text = []
    folder_name = os.path.basename(folder_path)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            text = text_from_image(image_path, textract_client)
            if text is not None:
                all_text.append({'Nombre_Carpeta': folder_name, 'Nombre_Imagen': filename, 'Texto_Extraido': text})

    if all_text:
        df = pd.DataFrame(all_text)
        parquet_file = os.path.join(output_path, f'{folder_name}.parquet')
        df.to_parquet(parquet_file, index=False)
        print(f"Datos guardados en {parquet_file}")
        return df
    return None

In [10]:
# Prueba de la función process_pdf_folder
#pdf_path = '/home/ec2-user/SageMaker/genebanks/data/2_images/Germplasm collection-Colombia-1990'
#output_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data'
#document = process_pdf_folder(pdf_path, textract_client, output_path)

In [11]:
#document

In [12]:
# Verificar parquet de salida
#parquet_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data/Germplasm collection-Colombia-1990.parquet'
#df = pd.read_parquet(parquet_path, engine='fastparquet')
#df

In [13]:
def process_folders(main_path, textract_client, output_path):
    all_dataframes = []
    for folder_name in os.listdir(main_path):
        folder_path = os.path.join(main_path, folder_name)
        if os.path.isdir(folder_path):
            df = process_pdf_folder(folder_path, textract_client, output_path)
            if df is not None:
                all_dataframes.append(df)

    if all_dataframes:
        combined_df = pd.concat(all_dataframes, ignore_index=True)
        combined_parquet_file = os.path.join(output_path, 'combined_data.parquet')
        combined_df.to_parquet(combined_parquet_file , index=False, engine='fastparquet')
        combined_csv_file = os.path.join(output_path, 'combined_data.csv')
        combined_df.to_csv(combined_csv_file, index=False)
        print(f"Datos guardados")

In [14]:
# Proceso completo
main_path = '/home/ec2-user/SageMaker/genebanks/data/2_images'
output_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data'

if not os.path.exists(output_path):
    os.makedirs(output_path)

process_folders(main_path, textract_client, output_path)

Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Recoleccion germoplasma-Peru-1977.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Germplasm exploration-Brasil-1980.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Informe Colecta Guatemala-1985.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Informe de colecta-Mexico1987.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Informe de colecta-Guatemala-1987.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Informe colecta Colombia-Risaralda-Caldas-1987.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Germplasm exploration-Malawi and Zambia-1980.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Germplasm exploration Costa Rica-1987.parquet
Datos guardados en /home/ec2-user/SageMaker/genebanks/data/3_out_data/Informe de co

NameError: name 'combined_file' is not defined